> **Pre-session prep:** Before opening this notebook, make sure students understand `pip install`, `getpass`, f-strings, and classes. See **[teaching-guide.md](teaching-guide.md)** for verbal talking points organized by teaching order.

<div align="center">
<img src="https://poorit.in/image.png" alt="Poorit" width="40" style="vertical-align: middle;"> <b>AI SYSTEMS ENGINEERING 1</b>

## Revision Session: Unit 1 + Unit 2 (Gradio)

**CV Raman Global University, Bhubaneswar**  
*AI Center of Excellence*

</div>

---

### Session Structure: 5 Acts in 120 Minutes

| Act | Topic | Analogy | Time |
|-----|-------|---------|------|
| 1 | API Calls | The Restaurant Order | 30 min |
| 2 | Tokens & Memory | The Memory Illusion | 25 min |
| 3 | JSON & Chaining | The Assembly Line | 20 min |
| 4 | Streaming | Live Cricket Commentary | 20 min |
| 5 | Gradio UIs | The Shop Counter | 25 min |

**Format:** I Do (watch) → We Do (predict) → You Do (code)

---

## Setup

Run the cells below to install packages and configure API keys.
This setup works in both Google Colab and local Jupyter.

In [ ]:
# Install required packages
!pip install -q openai tiktoken requests beautifulsoup4 gradio

In [ ]:
import os
import json
from getpass import getpass
from openai import OpenAI
from bs4 import BeautifulSoup
import requests
import tiktoken
import gradio as gr
from IPython.display import Markdown, display

In [ ]:
# Configure API Keys
openai_api_key = getpass("Enter your OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = openai_api_key

# OpenAI client
openai_client = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o-mini"
print(f"OpenAI configured with model: {MODEL}")

# Optional: Gemini client
google_api_key = getpass("Enter your Google API Key (or press Enter to skip): ")
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key) if google_api_key else None
if gemini_client:
    print("Gemini configured")
else:
    print("Gemini skipped (no key provided)")

# We'll use openai_client as the default
client = openai_client

In [ ]:
# Web scraping utility — reused across multiple acts

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url, max_chars=2000):
    """
    Fetch and return the title and text content of a website.
    Removes scripts, styles, and other non-text elements.
    """
    response = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(response.content, "html.parser")
    
    title = soup.title.string if soup.title else "No title found"
    
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    
    return (title + "\n\n" + text)[:max_chars]

print("Setup complete!")

---

# Act 1: The Restaurant Order — How LLM API Calls Work

**Covers:** Notebooks 1 + 2 | **Time:** 30 minutes

---

### The Analogy

Think of an LLM API call like ordering at a restaurant:

| Restaurant | Code |
|---|---|
| Which restaurant | `model="gpt-4o-mini"` |
| The waiter (takes your order) | `client = OpenAI()` |
| Your order slip | `messages=[{role, content}, ...]` |
| Chef's standing instructions | System prompt |
| Your actual request | User prompt |
| Food delivered | `response.choices[0].message.content` |
| The bill | Tokens (input + output) |
| Same waiter, different kitchen | `base_url` change for Gemini/Ollama |

---

### I Do: Your First API Call (Professor live codes)

In [ ]:
# Step 1: Create the messages list
# The API expects a list of messages, each with a "role" and "content"

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of Odisha?"}
]

# Step 2: Make the API call
response = client.chat.completions.create(
    model=MODEL,
    messages=messages
)

# Step 3: Extract the response
print(response.choices[0].message.content)

### We Do: What does each part do?

Before running the next cell, predict:
- What role should the system prompt have?
- What goes in the user message?
- Where is the response text stored?

In [ ]:
# Let's see what the raw response object looks like
print("Full response object:")
print(response)
print()
print("--- Breaking it down ---")
print(f"Model used: {response.model}")
print(f"Choices count: {len(response.choices)}")
print(f"Message role: {response.choices[0].message.role}")
print(f"Message content: {response.choices[0].message.content}")
print(f"Tokens used - Input: {response.usage.prompt_tokens}, Output: {response.usage.completion_tokens}")

### Why does content go in the user prompt (not system)?

The **system prompt** = chef's standing instructions ("always serve vegetarian")  
The **user prompt** = each specific order ("I want biryani")

When summarizing a website, the website text goes in the **user prompt** because it's the specific "order" — the content to process. The system prompt just says HOW to process it.

### You Do: System Prompt Swap

**Task:** Change the system prompt to make the model respond differently. Try:
1. First: Make it respond like a pirate
2. Then: Make it respond only in Hindi

In [ ]:
# YOUR TURN: Change the system prompt
# Try: "You are a pirate. Respond to everything in pirate speak."
# Or:  "You are a helpful assistant. Always respond in Hindi."

messages = [
    {"role": "system", "content": "___"},  # <-- Change this!
    {"role": "user", "content": "What is machine learning?"}
]

response = client.chat.completions.create(model=MODEL, messages=messages)
print(response.choices[0].message.content)

### I Do: Same Waiter, Different Kitchen (Provider Switch)

The OpenAI client library can talk to ANY provider that supports the same API format. We just change the `base_url` — like sending the same waiter to a different kitchen.

In [ ]:
# OpenAI (default kitchen)
print("--- OpenAI ---")
response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Tell me a fun fact about Bhubaneswar in one sentence."}]
)
print(response.choices[0].message.content)

# Gemini (different kitchen, same waiter!)
if gemini_client:
    print("\n--- Gemini ---")
    response = gemini_client.chat.completions.create(
        model="gemini-2.0-flash",
        messages=[{"role": "user", "content": "Tell me a fun fact about Bhubaneswar in one sentence."}]
    )
    print(response.choices[0].message.content)
else:
    print("\nGemini not configured — skipped")

### I Do: Client Library vs Raw HTTP (Quick Demo)

The client library is just a convenient wrapper around raw HTTP calls. Here's what it does under the hood:

In [ ]:
# What the OpenAI library does for you behind the scenes:
headers = {
    "Authorization": f"Bearer {openai_api_key}",
    "Content-Type": "application/json"
}

payload = {
    "model": "gpt-4o-mini",
    "messages": [{"role": "user", "content": "Say hello in one word"}]
}

raw_response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

# With raw HTTP, you get a dictionary — notice the different syntax:
print("Raw HTTP:", raw_response.json()["choices"][0]["message"]["content"])
# vs client library: response.choices[0].message.content

---

# Act 2: The Memory Illusion — Tokens & Statelessness

**Covers:** Notebook 3 | **Time:** 25 minutes

---

### Key Insight

LLMs have **zero memory**. Every API call is completely independent. When ChatGPT seems to "remember" your conversation, it's actually sending the **entire chat history** with every single message. That's why long conversations cost more!

---

### I Do: How LLMs See Your Text (Tokens)

In [ ]:
# tiktoken: OpenAI's tokenizer library
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# English vs Hindi — different token counts!
for text in ["Hello", "Namaste", "नमस्ते", "Bhubaneswar", "Machine Learning"]:
    tokens = encoding.encode(text)
    print(f"'{text}' → {len(tokens)} tokens  (IDs: {tokens})")

In [ ]:
# See what each token represents
text = "Hello, my name is Ravi"
tokens = encoding.encode(text)
print(f"Text: '{text}'")
print(f"Total tokens: {len(tokens)}")
print()
for token_id in tokens:
    print(f"  {token_id:6d} → '{encoding.decode([token_id])}'")

### I Do: The Memory Illusion (THE key demo)

Watch what happens when we make two separate API calls:

In [ ]:
# Call 1: Introduce ourselves
print("--- Call 1: Introducing ourselves ---")
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! My name is Priya!"}
]
response = client.chat.completions.create(model=MODEL, messages=messages)
call_1_reply = response.choices[0].message.content
print(f"Assistant: {call_1_reply}")

In [ ]:
# Call 2: Ask for our name — WITHOUT history
print("--- Call 2: Asking our name (NEW call, no history) ---")
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What's my name?"}
]
response = client.chat.completions.create(model=MODEL, messages=messages)
print(f"Assistant: {response.choices[0].message.content}")
print()
print("^^^ It DOESN'T remember! Each call is stateless.")

### We Do: Before running the next cell...

**Predict:** What will happen if we include the full conversation history in Call 3?  
**Think:** Why does this work? What does the `assistant` role do in the messages list?

In [ ]:
# Call 3: Ask for our name — WITH full history
print("--- Call 3: Asking our name (WITH history) ---")
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! My name is Priya!"},
    {"role": "assistant", "content": call_1_reply},  # Include what the model said before
    {"role": "user", "content": "What's my name?"}
]
response = client.chat.completions.create(model=MODEL, messages=messages)
print(f"Assistant: {response.choices[0].message.content}")
print()
print("^^^ Now it 'remembers'! We sent the full conversation.")

### Context Window & Cost

Everything must fit in the **context window**:

```
System Prompt + Conversation History + Current Message + Response
= Must all fit within the context window (e.g., 128K tokens for GPT-4o-mini)
```

**Cost formula:**
```
Cost = (input_tokens / 1,000,000) × input_price + (output_tokens / 1,000,000) × output_price
```

GPT-4o-mini: $0.15 per 1M input tokens, $0.60 per 1M output tokens

### You Do: Build the Memory Yourself

**Task:** Complete the 3 cells below to experience statelessness firsthand.

In [ ]:
# Step 1: Introduce yourself to the model
# Replace ___ with your name

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! My name is ___"}  # <-- Your name here
]

response = client.chat.completions.create(model=MODEL, messages=messages)
intro_reply = response.choices[0].message.content
print(f"Assistant: {intro_reply}")

In [ ]:
# Step 2: Ask your name WITHOUT history — observe the failure

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is my name?"}
]

response = client.chat.completions.create(model=MODEL, messages=messages)
print(f"Assistant: {response.choices[0].message.content}")
# It won't know your name!

In [ ]:
# Step 3: Now build the messages list WITH history — observe success
# Fill in the ___ parts

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "___", "content": "Hi! My name is ___"},        # <-- What role? What name?
    {"role": "___", "content": intro_reply},                   # <-- What role for the model's reply?
    {"role": "___", "content": "What is my name?"}             # <-- What role?
]

response = client.chat.completions.create(model=MODEL, messages=messages)
print(f"Assistant: {response.choices[0].message.content}")

---

# Act 3: The Assembly Line — JSON & Chaining

**Covers:** Notebook 4 concepts | **Time:** 20 minutes

---

### The Analogy

Think of a factory assembly line:

| Factory | Code |
|---|---|
| Station 1: Classifier worker | LLM Call 1 → JSON output |
| Labeled box between stations | `response_format={"type": "json_object"}` |
| Station 2: Fetcher worker | Your code calls `fetch_website_contents` |
| Station 3: Writer worker | LLM Call 2 generates pamphlet |

**Why JSON?** Because Station 2 is CODE, not a human. Code needs structured data it can parse reliably.

This is an early form of **Agentic AI** — LLM output decides what the code does next!

---

### I Do: JSON Mode (Professor live codes)

In [ ]:
# Ask the model to return structured data as JSON
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": 'Respond in JSON with keys: topic, summary, difficulty. Example: {"topic": "AI", "summary": "...", "difficulty": "beginner"}'},
        {"role": "user", "content": "Explain what web scraping is"}
    ],
    response_format={"type": "json_object"}   # <-- This forces JSON output
)

# The response is a JSON STRING — we need to parse it
raw_text = response.choices[0].message.content
print("Raw response (it's a string):")
print(raw_text)
print(f"Type: {type(raw_text)}")

print()

# json.loads() converts string → Python dictionary
result = json.loads(raw_text)
print("Parsed result (it's a dict now):")
print(f"Topic: {result['topic']}")
print(f"Difficulty: {result['difficulty']}")
print(f"Type: {type(result)}")

### We Do: What keys should we request?

If we wanted the model to analyze a college course, what JSON keys would be useful?

Think of 3-4 keys (e.g., `course_name`, `prerequisites`, ...)

In [ ]:
# Try it with the keys the class suggested!
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": 'Analyze the course and respond in JSON with keys: course_name, prerequisites, difficulty, career_paths'},
        {"role": "user", "content": "B.Tech in Computer Science with specialization in AI"}
    ],
    response_format={"type": "json_object"}
)

result = json.loads(response.choices[0].message.content)
print(json.dumps(result, indent=2))  # Pretty print

### I Do: Walking Through the Pipeline (Notebook 4 recap)

In Notebook 4, we built a pamphlet generator with this pipeline:

```
LLM Call 1: "Pick relevant links from this website" (returns JSON)
     ↓
Code: fetch_website_contents() for each selected link
     ↓  
LLM Call 2: "Write a pamphlet from this content" (returns markdown)
```

The LLM's output (selected links) **decides** what the code fetches next. That's the "agentic" part — the LLM is making decisions that drive the program.

---

# Act 4: Live Cricket Commentary — Streaming & Generators

**Covers:** Notebook 4 streaming + Python generators | **Time:** 20 minutes

---

### The Analogy

| Cricket | Code |
|---|---|
| Full match highlights (wait, then watch all) | Normal API call (synchronous) |
| Ball-by-ball commentary (live!) | `stream=True` |
| Each ball delivery | `chunk.choices[0].delta.content` |
| Scoreboard updating live | `yield` to UI |
| `return` = full highlights DVD | Returns one complete value |
| `yield` = live commentary mic | Returns values one at a time (generator) |

---

### I Do: return vs yield (Professor demo)

In [ ]:
# RETURN: Gives you the full result at the end (match highlights)
def get_commentary_return():
    commentary = ""
    for ball in ["Ball 1: Dot ball. ", "Ball 2: Four! ", "Ball 3: Six! ", "Ball 4: Wicket! "]:
        commentary += ball
    return commentary  # You get EVERYTHING at once, at the end

result = get_commentary_return()
print("With return (all at once):")
print(result)

In [ ]:
import time

# YIELD: Gives you each piece as it happens (live commentary)
def get_commentary_yield():
    commentary = ""
    for ball in ["Ball 1: Dot ball. ", "Ball 2: Four! ", "Ball 3: Six! ", "Ball 4: Wicket! "]:
        commentary += ball
        yield commentary  # Send the current state RIGHT NOW

print("With yield (one at a time):")
for update in get_commentary_yield():
    print(update)
    time.sleep(0.5)  # Simulate delay between balls

### I Do: Streaming from the API

In [ ]:
# Non-streaming: wait... wait... then get everything
print("--- Non-streaming (match highlights) ---")
response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "Write a 3-line poem about cricket."}]
)
print(response.choices[0].message.content)  # .message.content for normal

In [ ]:
# Streaming: text flows in word by word!
print("--- Streaming (live commentary) ---")
stream = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "Write a 3-line poem about cricket."}],
    stream=True  # <-- This is the only change!
)

for chunk in stream:
    content = chunk.choices[0].delta.content or ""  # .delta.content for streaming
    print(content, end="", flush=True)

### You Do: Add Streaming to a Working Call

**Task:** The cell below has a working non-streaming API call. Convert it to streaming by:
1. Adding `stream=True`
2. Writing the chunk loop
3. Printing each chunk as it arrives

Fill in the `___` parts:

In [ ]:
# YOUR TURN: Convert this to streaming

# This is the non-streaming version (works but waits):
# response = client.chat.completions.create(
#     model=MODEL,
#     messages=[{"role": "user", "content": "Explain AI in 3 sentences."}]
# )
# print(response.choices[0].message.content)

# Convert to streaming — fill in the blanks:
stream = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "Explain AI in 3 sentences."}],
    ___=True  # <-- What parameter enables streaming?
)

for ___ in stream:  # <-- What variable holds each piece?
    content = ___.choices[0].___.content or ""  # <-- .message or .delta?
    print(content, end="", flush=True)

---

# Act 5: The Shop Counter — Gradio UIs

**Covers:** Unit 2 Gradio notebooks | **Time:** 25 minutes

---

### The Analogy

| Shop | Code |
|---|---|
| Your cooking function | Python function |
| Counter setup | `gr.Interface(fn, inputs, outputs)` |
| Menu board with examples | `examples=[...]` |
| Fancy restaurant with order history | `gr.ChatInterface(fn, type="messages")` |
| Waiter's notepad | `history` parameter |
| Adjusting recipe based on customer | Dynamic system prompts |

---

### I Do: Step 1 — Simplest Gradio App

In [ ]:
# Step 1: Any Python function can become a web app
def shout(text):
    return text.upper()

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

### I Do: Step 2 — Connect to an LLM

In [ ]:
# Step 2: Replace our simple function with an LLM call
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Respond in markdown."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.Interface(
    fn=message_gpt,
    inputs=gr.Textbox(label="Your message:", lines=3),
    outputs=gr.Markdown(label="Response:"),
    title="GPT Assistant",
    examples=["Explain recursion simply", "What is the capital of Odisha?"],
    flagging_mode="never"
).launch()

### I Do: Step 3 — Add Streaming with yield

In [ ]:
# Step 3: Add streaming — just change return to yield!
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Respond in markdown."},
        {"role": "user", "content": prompt}
    ]
    stream = client.chat.completions.create(model=MODEL, messages=messages, stream=True)
    
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result  # Gradio picks up each yield and updates the UI!

gr.Interface(
    fn=stream_gpt,
    inputs=gr.Textbox(label="Your message:", lines=3),
    outputs=gr.Markdown(label="Response:"),
    title="GPT Assistant (Streaming)",
    examples=["Write a poem about coding", "Explain neural networks"],
    flagging_mode="never"
).launch()

### I Do: Step 4 — Add a Dropdown for Options

In [ ]:
# Step 4: Add a dropdown to choose tone
def stream_with_tone(prompt, tone):
    system_prompts = {
        "Formal": "You are a formal, professional assistant. Respond in markdown.",
        "Funny": "You are a witty, humorous assistant. Use jokes and puns. Respond in markdown.",
        "Simple": "You are an assistant that explains things as simply as possible, like to a 10-year-old. Respond in markdown."
    }
    
    messages = [
        {"role": "system", "content": system_prompts[tone]},
        {"role": "user", "content": prompt}
    ]
    stream = client.chat.completions.create(model=MODEL, messages=messages, stream=True)
    
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

gr.Interface(
    fn=stream_with_tone,
    inputs=[
        gr.Textbox(label="Your message:", lines=3),
        gr.Dropdown(["Formal", "Funny", "Simple"], label="Tone", value="Formal")
    ],
    outputs=gr.Markdown(label="Response:"),
    title="Multi-Tone Assistant",
    examples=[
        ["Explain machine learning", "Formal"],
        ["Explain machine learning", "Funny"],
        ["Explain machine learning", "Simple"]
    ],
    flagging_mode="never"
).launch()

### I Do: ChatInterface (Conversations with Memory)

In [ ]:
# gr.ChatInterface handles history automatically!
def chat(message, history):
    # Convert Gradio's history format to OpenAI's format
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Build the full messages list: system + history + current message
    messages = (
        [{"role": "system", "content": "You are a helpful assistant."}]
        + history
        + [{"role": "user", "content": message}]
    )
    
    # Stream the response
    stream = client.chat.completions.create(model=MODEL, messages=messages, stream=True)
    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

gr.ChatInterface(fn=chat, type="messages").launch()

### You Do: Build a Gradio App (Culminating Exercise)

**Task:** Build a Gradio app that:
1. Takes a **topic** (textbox) and a **tone** (dropdown: Formal / Funny / Simple)
2. Generates an explanation of the topic in the selected tone
3. Uses **streaming** (yield, not return)

Fill in the `___` parts below. This tests everything from Acts 1-5!

In [ ]:
# YOUR TURN: Build a complete Gradio app

def explain_topic(topic, tone):
    # Step 1: Choose system prompt based on tone
    if tone == "Formal":
        system = "You are a formal professor. Explain topics clearly and professionally. Respond in markdown."
    elif tone == "Funny":
        system = "You are a comedian who explains topics with humor and jokes. Respond in markdown."
    else:
        system = "You explain topics as simply as possible, like to a 10-year-old. Respond in markdown."
    
    # Step 2: Create the messages list
    messages = [
        {"role": "___", "content": system},             # <-- What role?
        {"role": "___", "content": f"Explain: {topic}"}  # <-- What role?
    ]
    
    # Step 3: Make the streaming API call
    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        ___=True  # <-- What parameter enables streaming?
    )
    
    # Step 4: Yield results as they arrive
    result = ""
    for chunk in stream:
        result += chunk.choices[0].___.content or ""  # <-- .message or .delta?
        ___ result  # <-- return or yield?

# Step 5: Create the Gradio interface
gr.Interface(
    fn=___,  # <-- Which function?
    inputs=[
        gr.Textbox(label="Topic", lines=2),
        gr.Dropdown(["Formal", "Funny", "Simple"], label="Tone", value="Formal")
    ],
    outputs=gr.Markdown(label="Explanation"),
    title="Topic Explainer",
    flagging_mode="never"
).launch()

---

## The Big Picture

Here's the full journey you've completed:

```
Notebook 1: API calls + Web scraping
     ↓
Notebook 2: Multiple providers (same interface!)
     ↓
Notebook 3: Tokens, costs, memory illusion
     ↓
Notebook 4: JSON + Chaining + Streaming
     ↓
Unit 2: Gradio UIs (Interface + ChatInterface)
```

### Cross-cutting patterns that appear EVERYWHERE:

1. **The API call pattern** — same `client.chat.completions.create()` in every notebook
2. **`fetch_website_contents`** — reused across notebooks 1, 4, and Gradio
3. **Progression**: synchronous → streaming → streaming in Gradio UI
4. **The wrapper function pattern** — every notebook wraps lower-level functions into higher-level ones

### You can now:
- Call any LLM (OpenAI, Gemini, local via Ollama)
- Scrape and process web content
- Chain multiple LLM calls into pipelines
- Stream responses for better UX
- Build interactive UIs with Gradio

**That's AI Systems Engineering.**

---

**Course Information:**
- **Institution:** CV Raman Global University, Bhubaneswar
- **Program:** AI Center of Excellence
- **Course:** AI Systems Engineering 1
- **Developed by:** [Poorit Technologies](https://poorit.in) - *Transform Graduates into Industry-Ready Professionals*

---